# 🚀 OpenAI Python SDK 101

In this notebook we’ll learn how to interact with Large Language Models (LLMs) directly using the **OpenAI Python SDK**.  
This is the **first time** we’re exploring API interactions, so we’ll build up gradually:

1. **Initialize** the client with your API key.  
2. **Minimal call** to the API (Responses API).  
3. Use **Chat Completions** for system + user roles.  
4. Explore **temperature** (randomness) and **top_p** (nucleus sampling).  
5. Add **system prompts** to guide behavior.  
6. Try **streaming tokens** (like ChatGPT typing).  
7. Get **JSON/structured outputs** with schemas.  
8. Handle **errors, timeouts, and retries** gracefully.

By the end, you’ll know how to **call an LLM safely and flexibly** using just the OpenAI SDK.

In [4]:
import os, json, time
from typing import Any, Dict

### API key
- Set your OpenAI API key as an environment variable:  
  `export OPENAI_API_KEY="sk-..."` (macOS/Linux) or `setx OPENAI_API_KEY "sk-..."` (Windows, new terminal required).  
- In Colab: use `os.environ["OPENAI_API_KEY"] = "..."` (for demos only).


In [5]:
# OpenAI Python SDK v1 style
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Minimal "Responses API" call (recommended by OpenAI for new projects)
# Docs: https://platform.openai.com/docs/guides/text  and Responses vs Chat Completions
resp = client.responses.create(
    model="gpt-4o-mini",  # choose any available text-capable model
    input="In one sentence, explain the difference between temperature and top_p for sampling."
)
print(resp.output_text)

Temperature controls the randomness of predictions by scaling the probabilities of possible outcomes, while top_p (nucleus sampling) limits the selection to a subset of the most probable options, ensuring a balance between randomness and coherence in the generated text.


In [6]:
# Using Chat Completions (still widely used & supported)
chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a concise teaching assistant."},
        {"role": "user", "content": "Give me 3 bullet points about overfitting."}
    ]
)
print(chat.choices[0].message.content)

- **Definition**: Overfitting occurs when a machine learning model learns the noise or random fluctuations in the training data rather than the underlying pattern, leading to poor generalization to new, unseen data.

- **Symptoms**: Indicators of overfitting include a model that performs significantly better on training data compared to validation/test data, often showing high accuracy on the former and low accuracy on the latter.

- **Solutions**: Techniques to mitigate overfitting include using simpler models (regularization), reducing the number of features, employing cross-validation, and using techniques like dropout in neural networks.


In [7]:
messages = [
    {"role": "system", "content": "You are a Python tutor who answers with short code examples."},
    {"role": "user", "content": "Show how to reverse a string in Python."}
]
r = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0.2)
print(r.choices[0].message.content)

You can reverse a string in Python using slicing. Here's a simple example:

```python
original_string = "Hello, World!"
reversed_string = original_string[::-1]
print(reversed_string)
```

This will output:

```
!dlroW ,olleH
```


In [10]:
from sys import stdout

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Write a short story about a cat and a dog."}],
    temperature=0.7,
    stream=True,
)

for event in stream:
    if hasattr(event, "choices"):
        delta = event.choices[0].delta
        if delta and delta.content:
            stdout.write(delta.content)
stdout.write("\n")

In a quiet little town nestled between rolling hills, there lived a cat named Whiskers and a dog named Max. Though they lived in neighboring houses, they had never met. Whiskers was a sleek, silver tabby with piercing green eyes, known for her grace and independence. Max, on the other hand, was a big, fluffy golden retriever with a heart as big as his bark, always eager for companionship and adventure.

One sunny afternoon, Whiskers decided to explore beyond her usual territory. She ventured into the backyard of Max's home, drawn by the tantalizing scent of blooming flowers and the sound of a rustling breeze. As she crept through the tall grass, she spotted Max chasing a butterfly, his tail wagging furiously.

Curious, Whiskers perched herself on a low fence post to watch. Max, noticing her, skidded to a stop, panting with excitement. "Hello there! I’m Max! Want to play?" he barked, his voice filled with enthusiasm.

Whiskers flicked her tail, unsure. “Play? With you? I’m not so sure a

1

In [ ]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI


class Summary(BaseModel):
    topic: str
    key_points: List[str]


client = OpenAI()

completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    temperature=0,
    response_format=Summary,
    messages=[
        {"role": "user", "content": "Topic: Transformers in NLP. Give 3 key points."}
    ],
)

parsed = completion.choices[0].message.parsed
parsed

Summary(topic='Transformers in NLP', key_points=['**Revolutionary Architecture:** \nTransformers have revolutionized the field of Natural Language Processing (NLP) by introducing a novel architecture that relies on self-attention mechanisms. This allows models to weigh the importance of different words in a sentence, capturing long-range dependencies more effectively than previous models like RNNs and LSTMs.', '**Scalability and Efficiency:** \nTransformers are highly scalable and can be trained on large datasets, making them suitable for a wide range of NLP tasks. Their parallelizable structure allows for efficient training on modern hardware, leading to faster processing times and the ability to handle vast amounts of data.', '**Foundation for Pre-trained Models:** \nTransformers serve as the backbone for many state-of-the-art pre-trained language models, such as BERT, GPT, and T5. These models leverage the transformer architecture to achieve high performance on various NLP tasks, in